# 仮説

* ユーザーの行動に流れがあるのではないか？
* 商品の情報も用いてさらにユーザーを分類することはできないか？  
人気商品について、買う人買わない人がいそう？

# 評価値　仮説

* クラスターごとに評価値設定の仕方を変えられるのではないか？
    * １か月間cvをしなかったクラスターは将来的にもcvをしないのではないか？→カートから閲覧を予測(相関をみよう)
    * 購入する回数が多いクラスターは、何かしらの購入までの流れを加味できないか？
    * 閲覧したけど買わなかったら低評価？高評価？　１回閲覧して複数回してなかったら低評価？
* 人気商品に敏感なクラスターがあるのでは？


* 評価値を出した後、そのユーザーの最大評価値ですべての評価値を割り算して、相対的な評価値にした方がよい？

# 推薦　仮説

* 直前に見ていたもの＋それらに類似する商品を推薦する
* 一度買ったものはもう買わないとする処理がいる←oginoさんが実装済み？
* 一度閲覧したものは推薦しない
* 類似したユーザーの評価値が高いものを推薦する

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from IPython.core.display import display
import seaborn as sns
%matplotlib inline

In [2]:
user_df = pd.read_csv("../..data/train/clustered_user_with_cv_D.csv")
product_df = pd.read_csv("../../data/train/clustered_product_without_cv_D.csv")

In [2]:
df = pd.read_csv("../../data/train/train_D.tsv", sep="\t")
df["time_stamp"] = pd.to_datetime(df["time_stamp"])
user_ids = df["user_id"].unique()
product_ids = df["product_id"].unique()

In [4]:
p_product_df = product_df[(product_df["cluster"] == 2) | (product_df["cluster"] == 3)]
np_product_df = product_df[(product_df["cluster"] == 0) | (product_df["cluster"] == 1)]
p_product_ids = p_product_df["product_id"].unique()
np_product_ids = np_product_df["product_id"].unique()

In [5]:
buyer_ids = df[df["event_type"]==3]["user_id"].unique()

In [6]:
display(user_df.head(3))
display(product_df.head(3))

,user_id,ca,pd,cv,cluster
0,0000000_D,23.0,5.0,0.0,5
1,0000001_D,32.0,18.0,0.0,8
2,0000002_D,4.0,6.0,0.0,4


,product_id,ca,pd,cv,cluster
0,00000000_d,6.0,3.0,0.0,0
1,00000001_d,1.0,3.0,0.0,0
2,00000002_d,0.0,2.0,0.0,0


# 商品

# ユーザー

In [8]:
cluster_len = len(user_df.cluster.unique())

In [70]:
user_df.groupby("cluster").size()

cluster
0         5
1        23
2        28
3       407
4     48230
5     34592
6         2
7       358
8     20225
9       306
10     4431
dtype: int64

* 0  
cartが多(mean:10538)・pdが少(mean:19.4)・cvは0  
* 1  
cart(mean:70)・pd(mean:15)・**cvを全員持つ(mean:9.5)**  
* 2  
cartが多(mean:3192)・pdが少(mean:23)・cvは0
* 3  
cart(mean:132)・pd(mean:77)・cvは0・閲覧を全員50以上する  
* 4  
cartも少(mean:12)・pdも少(mean:9)・cvは0  
* 5  
cartも少(mean:25)・pdも少(mean:3)・cvは0  
* 6  
cartが少(2,192)・pdが多(1530,1144)・**cvもあったりなかったり(0,17)**
* 7  
cart(mean:23)・pd(mean:13)・**cvを全員持つ(mean:1.5)**  
* 8  
cart(mean:21)・pd(mean:16)・cvは0  
* 9  
cartが多(mean:615)・pdが少(mean:14)・cvは0  
* 10  
cart(mean:50)・pd(mean:33)・cvは0・閲覧を全員20以上する  

## 評価値 仮説
* Cluster0
    * カートに入れることしかしていないユーザー
        - カート回数が多いほど評価を（上げる・）下げる（微小量ずつ）←商品のカート数と閲覧数に相関ほとんどなし（マイナス）
    * 閲覧もしているユーザー
        * カートに入れ、かつ閲覧をしているもの・・・評価値をあげる
        * カートに入れるだけ・・・評価値を（上げる・）下げる（微小量だけ）
        * 閲覧だけ・・・評価値を大きく上げる
* Cluster2,9
    * カートに入れることしかしていないユーザー
        - カート回数が多いほど評価を上げる（・下げる）（微小量ずつ）←商品のカート数と閲覧数に相関ほとんどなし
    * 閲覧もしているユーザー
        * カートに入れ、かつ閲覧をしているもの・・・評価値を大きくあげる
        * カートに入れるだけ・・・評価値を上げる（・下げる）（微小量だけ）
        * 閲覧だけ・・・評価値を上げる
-----
* Cluster4  
    * カートと閲覧の回数が多いほど評価を上げる(カートに関しては微小量)←商品のカート数と閲覧数に相関ほとんどなし
* Cluster5,8
    * カートと閲覧の回数が多いほど評価を上げる(カートに関してはCluster4よりは大きく)←商品のカート数と閲覧数に相関あり
-----
* Cluster1,7  
    * 買った商品・・・評価値上げる（ただし、推薦はしない）
    * 買う前（2,3日前）に見てたけど買わなかった商品・・・評価値下げる 
    * カートと閲覧の回数が多いほど評価を上げる(カートに関しては微小量)
-----
* Cluster3  
    * カート回数が多いほど評価を下げる(微小量) 
* Cluster10  
    * カートの回数が多いほど評価を下げる(Cluster3よりは大きく)
* 共通
    * 閲覧した商品・・・評価値上げる（ただし、推薦はしない）
    * 閲覧前（2,3日前）にカートにいれてたけど買わなかった商品・・・評価値下げる （仮）
    
-----
* Cluster6 
    * 購入経験のあるユーザー（0016920_D）
    * 購入経験のないユーザー（0102238_D）

商品に関してはcaが多いほどpdも多い

In [18]:
def describe_cluster(i, df):
    print("cluster_" + str(i))
    display_df = pd.DataFrame()
    display_target_df = df[df["cluster"] == i][["ca","pd","cv"]]
    if display_df.empty:
        display_df = display_target_df.describe().T
    else:
        display_df = pd.concat([display_df, display_target_df.describe().T])
    display(display_df)
    print()

# for i in range(0,11):
#     describe_cluster(i, user_df)

In [7]:
user_count_dfs = []
for i in range(cluster_len):
    user_cluster_df = user_df[user_df["cluster"] == i]
    user_count_dfs.append(user_cluster_df)

In [15]:
df_added_cluster = pd.merge(df, user_df[["user_id", "cluster"]], on='user_id')

In [19]:
df_added_cluster[df_added_cluster["cluster"]==0]

array(['0041349_D', '0042222_D', '0014163_D', '0043330_D', '0100726_D'], dtype=object)

In [21]:
for i in range(cluster_len):
    df_added_cluster[df_added_cluster["cluster"]==i].to_csv("../../data/train/D_cluster_" + str(i) + ".csv",index=False)

In [4]:
def make_count_df_p(df):
    dfs = []
    labels = ["ca", "pd", "cl", "cv"]
    for i, label in enumerate(labels):
        dic = {"id":i, "label":label, "df":df[df["event_type"] == i]}
        dfs.append(dic)
    c_dfs= []
    for df_item in dfs:
        new_df = pd.DataFrame(df_item["df"].groupby("product_id").size(), columns=[df_item["label"]])
        dic = {"id":df_item["id"], "label":df_item["label"], "df":new_df}
        c_dfs.append(dic)
    count_df = pd.concat([c_dfs[0]["df"],c_dfs[1]["df"],c_dfs[3]["df"]], axis=1).fillna(0)
    return count_df

In [5]:
def make_count_df_u(df):
    dfs = []
    labels = ["ca", "pd", "cl", "cv"]
    for i, label in enumerate(labels):
        dic = {"id":i, "label":label, "df":df[df["event_type"] == i]}
        dfs.append(dic)
    c_dfs= []
    for df_item in dfs:
        new_df = pd.DataFrame(df_item["df"].groupby("user_id").size(), columns=[df_item["label"]])
        dic = {"id":df_item["id"], "label":df_item["label"], "df":new_df}
        c_dfs.append(dic)
    count_df = pd.concat([c_dfs[0]["df"],c_dfs[1]["df"],c_dfs[3]["df"]], axis=1).fillna(0)
    return count_df

In [12]:
def get_user_ids(cluster, user_df):
    user_ids = list(user_df[user_df["cluster"] == cluster].user_id)
    return user_ids

In [7]:
def get_action_df(cluster_df):
    action_df = cluster_df.drop_duplicates(["user_id","product_id","event_type"]).sort_values(by=["time_stamp"])
    return action_df

In [9]:
# cluster_user_ids = []
# for i in range(cluster_len):
#     user_ids = get_user_ids(i, user_df)
#     cluster_user_ids.append(user_ids)

In [85]:
def prepare_cluster_analysis(cluster, user_df):
    filename = "../../data/train/D_cluster_{}.csv".format(cluster)
    cluster_df = pd.read_csv(filename)
    action_df = get_action_df(cluster_df)
    user_ids = get_user_ids(cluster, user_df)
    return cluster_df, action_df, user_ids

def count_analysis(cluster_df):
    count_df_p = make_count_df_p(cluster_df)
    count_df_u = make_count_df_u(cluster_df)
    print("product")
    display(count_df_p.corr())
    print("user")
    display(count_df_u.corr())

### Cluster 0
cartが多(mean:10538)・pdが少(mean:19.4)・cvは0

* 0014163_D, 0041349_D, 0042222_D  
【評価値】
カートに入れ、かつ閲覧をしているもの→高く  
カートに入れただけ→前者よりは低く  
閲覧しただけ→前者よりは低く  

最終カート入れ・閲覧時が前すぎたら推薦しない（評価値は下げずに←興味はあっても買わなかっただけかも←閲覧くらいしてくれるかも）

* 0043330_D,0100726_D  
閲覧履歴がないため、おそらく閲覧はしないものと思われるため、後回し。

In [106]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(0, user_df)
user_ids

['0014163_D', '0041349_D', '0042222_D', '0043330_D', '0100726_D']

In [41]:
user_df[user_df["cluster"] == 0]

,user_id,ca,pd,cv,cluster
14163,0014163_D,7887.0,9.0,0.0,0
41349,0041349_D,8944.0,34.0,0.0,0
42222,0042222_D,8934.0,54.0,0.0,0
43330,0043330_D,12165.0,0.0,0.0,0
100726,0100726_D,14761.0,0.0,0.0,0


In [109]:
action_df[(action_df["user_id"]=="0014163_D")&(action_df["time_stamp"]>="2017-04-25")].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
25696,0014163_D,00406357_d,0,-1,2017-04-25 03:46:05.085,0
25693,0014163_D,00561560_d,0,-1,2017-04-25 08:42:40.010,0
25697,0014163_D,00555258_d,0,-1,2017-04-25 08:51:26.534,0
25669,0014163_D,00297783_d,0,-1,2017-04-25 08:52:00.326,0
18655,0014163_D,00398893_d,0,-1,2017-04-25 10:06:02.895,0
18175,0014163_D,00683199_d,0,-1,2017-04-25 11:17:45.841,0
25710,0014163_D,00195072_d,0,-1,2017-04-25 11:55:11.108,0
25656,0014163_D,00711572_d,0,-1,2017-04-25 12:41:44.109,0
25677,0014163_D,00576826_d,0,-1,2017-04-25 13:50:33.420,0
25725,0014163_D,00181525_d,0,-1,2017-04-25 16:39:25.590,0


In [69]:
for user_id in user_ids:
    print(user_id)
    for p_id in list(action_df[(action_df["user_id"]==user_id) & (action_df["event_type"]==1)].product_id):
        display(cluster_df[(cluster_df.event_type == 1) & (cluster_df["user_id"]==user_id) & (cluster_df["product_id"]==p_id)])

0014163_D


,user_id,product_id,event_type,ad,time_stamp,cluster
19045,0014163_D,00201404_d,1,-1,2017-04-02 15:00:27.120,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18943,0014163_D,00225045_d,1,-1,2017-04-05 17:28:26.426,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18626,0014163_D,00220513_d,1,-1,2017-04-06 16:16:34.523,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18070,0014163_D,00338331_d,1,-1,2017-04-13 15:10:33.529,0


...

,user_id,product_id,event_type,ad,time_stamp,cluster
10947,0042222_D,00237978_d,1,-1,2017-04-27 10:18:03.198,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9199,0042222_D,00542183_d,1,-1,2017-04-27 14:11:45.125,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9140,0042222_D,00200638_d,1,-1,2017-04-27 16:48:26.905,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9251,0042222_D,00200353_d,1,-1,2017-04-27 22:38:04.664,0


0043330_D
0100726_D


In [71]:
for user_id in user_ids:
    print(user_id)
    for p_id in list(action_df[(action_df["user_id"]==user_id) & (action_df["event_type"]==1)].product_id):
        display(action_df[(action_df["user_id"]==user_id) & (action_df["product_id"]==p_id)])

0014163_D


,user_id,product_id,event_type,ad,time_stamp,cluster
19045,0014163_D,00201404_d,1,-1,2017-04-02 15:00:27.120,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18943,0014163_D,00225045_d,1,-1,2017-04-05 17:28:26.426,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18626,0014163_D,00220513_d,1,-1,2017-04-06 16:16:34.523,0
18627,0014163_D,00220513_d,0,-1,2017-04-06 23:26:52.043,0


,user_id,product_id,event_type,ad,time_stamp,cluster
18070,0014163_D,00338331_d,1,-1,2017-04-13 15:10:33.529,0


...

,user_id,product_id,event_type,ad,time_stamp,cluster
10947,0042222_D,00237978_d,1,-1,2017-04-27 10:18:03.198,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9200,0042222_D,00542183_d,0,-1,2017-04-26 21:53:03.369,0
9199,0042222_D,00542183_d,1,-1,2017-04-27 14:11:45.125,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9141,0042222_D,00200638_d,0,-1,2017-04-27 16:48:09.630,0
9140,0042222_D,00200638_d,1,-1,2017-04-27 16:48:26.905,0


,user_id,product_id,event_type,ad,time_stamp,cluster
9252,0042222_D,00200353_d,0,-1,2017-04-27 14:22:58.881,0
9251,0042222_D,00200353_d,1,-1,2017-04-27 22:38:04.664,0


0043330_D
0100726_D


In [73]:
count_df_p = make_count_df_p(cluster_df)

In [74]:
count_df_p.corr()

,ca,pd,cv
ca,1.000000,-0.031361,NaN
pd,-0.031361,1.000000,NaN
cv,NaN,NaN,NaN


In [76]:
count_df_u = make_count_df_u(cluster_df)

In [77]:
count_df_u.corr()

,ca,pd,cv
ca,1.000000,-0.605039,NaN
pd,-0.605039,1.000000,NaN
cv,NaN,NaN,NaN


In [11]:
describe_cluster(0, user_df)

cluster_0


,count,mean,std,min,25%,50%,75%,max
ca,5.0,10538.2,2855.797209,7887.0,8934.0,8944.0,12165.0,14761.0
pd,5.0,19.4,23.828554,0.0,0.0,9.0,34.0,54.0
cv,5.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


### Cluster 1
cart(mean:70)・pd(mean:15)・**cvを全員持つ(mean:9.5)**

同じ商品に関して、0=>1>=3のような流れがほとんどない。もはやいきなり購入してる。  
→おそらく似たような商品を複数ざっと見て、その中から選んでる？  
→買う前にカートに入れたりしている商品は、買った商品と似ている商品なのではないか？
* 買った商品・・・評価値上げる（ただし、推薦はしない）
* 買う前（2,3日前）に見てたけど買わなかった商品・・・評価値下げる
* 買う前にかなり見てたけど買わなかった商品・・・評価値少し上げる。  
☆買った後のカートの意味がわからない

In [81]:
cluster_df = pd.read_csv("../../data/train/D_cluster_1.csv")
action_df = get_action_df(cluster_df)
user_ids = get_user_ids(1, user_df)
user_ids[0:5]

['0006278_D', '0010267_D', '0013712_D', '0014405_D', '0014834_D']

In [82]:
action_df[action_df["user_id"]=="0105538_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
1644,0105538_D,00155860_d,1,-1,2017-04-04 08:31:40.929,1
1634,0105538_D,00011390_d,1,-1,2017-04-04 09:05:43.364,1
1648,0105538_D,00155903_d,1,-1,2017-04-04 09:42:29.556,1
1652,0105538_D,00061060_d,1,-1,2017-04-04 13:52:16.351,1
1635,0105538_D,00772716_d,1,-1,2017-04-04 14:47:34.085,1
1656,0105538_D,00578101_d,1,-1,2017-04-05 08:55:35.561,1
1661,0105538_D,00396712_d,0,-1,2017-04-05 10:29:54.162,1
1650,0105538_D,00007227_d,1,-1,2017-04-05 12:59:00.629,1
1657,0105538_D,00604822_d,1,-1,2017-04-06 02:55:54.517,1
1636,0105538_D,00460621_d,1,-1,2017-04-06 09:13:41.007,1


In [83]:
describe_cluster(1, user_df)

cluster_1


,count,mean,std,min,25%,50%,75%,max
ca,23.0,70.260870,106.974176,1.0,8.5,16.0,66.5,341.0
pd,23.0,15.304348,15.516662,0.0,7.5,11.0,19.0,61.0
cv,23.0,9.478261,6.960078,6.0,6.5,7.0,9.0,39.0


In [86]:
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,-0.142294,-0.084942
pd,-0.142294,1.000000,-0.084321
cv,-0.084942,-0.084321,1.000000


user


,ca,pd,cv
ca,1.000000,0.142129,-0.072275
pd,0.142129,1.000000,-0.165134
cv,-0.072275,-0.165134,1.000000


### Cluster7

In [124]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(7, user_df)
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,-0.239099,-0.014707
pd,-0.239099,1.000000,-0.037761
cv,-0.014707,-0.037761,1.000000


user


,ca,pd,cv
ca,1.000000,0.336527,0.217254
pd,0.336527,1.000000,0.224776
cv,0.217254,0.224776,1.000000


### Cluster2
cartが多(mean:3192)・pdが少(mean:23)・cvは0

In [87]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(2, user_df)
user_ids[0:5]

['0002974_D', '0005907_D', '0009793_D', '0011704_D', '0012042_D']

In [88]:
action_df[action_df["user_id"]=="0002974_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
30654,0002974_D,00150124_d,0,-1,2017-03-31 15:26:10.136,2
30692,0002974_D,00490642_d,0,-1,2017-03-31 15:26:10.136,2
30461,0002974_D,00068140_d,0,-1,2017-04-01 01:36:17.443,2
30696,0002974_D,00066754_d,0,-1,2017-04-01 02:45:47.634,2
30465,0002974_D,00390308_d,0,-1,2017-04-01 03:02:06.572,2
30312,0002974_D,00450321_d,0,-1,2017-04-01 03:03:24.349,2
30360,0002974_D,00788776_d,0,-1,2017-04-01 03:03:24.349,2
30614,0002974_D,00042441_d,0,-1,2017-04-01 03:03:56.796,2
30269,0002974_D,00091528_d,0,-1,2017-04-01 03:04:37.110,2
30483,0002974_D,00069358_d,0,-1,2017-04-01 03:17:01.225,2


In [58]:
describe_cluster(2, user_df)

cluster_2


,count,mean,std,min,25%,50%,75%,max
ca,28.0,3191.035714,1139.957552,1923.0,2467.5,2695.0,3629.25,5956.0
pd,28.0,23.428571,39.226246,0.0,0.0,10.0,21.75,180.0
cv,28.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0


In [89]:
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.015076,NaN
pd,0.015076,1.000000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,-0.131537,NaN
pd,-0.131537,1.000000,NaN
cv,NaN,NaN,NaN


### Cluster9

In [110]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(9, user_df)
user_ids[0:5]

['0000368_D', '0000411_D', '0000930_D', '0001000_D', '0001241_D']

In [111]:
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.00000,0.03475,NaN
pd,0.03475,1.00000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,0.105469,NaN
pd,0.105469,1.000000,NaN
cv,NaN,NaN,NaN


### Cluster4,5,8
4 cartも少(mean:12)・pdも少(mean:9)・cvは0 /48230人  
5 cartも少(mean:25)・pdも少(mean:3)・cvは0 /34592人  
8 cart(mean:21)・pd(mean:16)・cvは0 /20225人

* カートと閲覧の回数が多いほど評価を上げる  
    - １ユーザー当たりのデータ数が少ない。
    - 購入経験がない→カート・閲覧から購入という流れがない。
    - カートから閲覧という流れもない。→良い評価値設定の方法が難しい。

In [59]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(4, user_df)
user_ids[0:10]

['0000002_D',
 '0000004_D',
 '0000012_D',
 '0000014_D',
 '0000016_D',
 '0000017_D',
 '0000018_D',
 '0000020_D',
 '0000024_D',
 '0000025_D']

In [61]:
action_df[action_df["user_id"]=="0000004_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
45973,0000004_D,00663924_d,1,-1,2017-04-02 08:27:54.732,4
45974,0000004_D,00217794_d,1,-1,2017-04-02 08:43:37.222,4
45981,0000004_D,00439308_d,0,-1,2017-04-02 10:33:55.399,4
45975,0000004_D,00502967_d,1,-1,2017-04-09 11:53:10.215,4
45976,0000004_D,00417270_d,1,-1,2017-04-15 13:53:51.291,4
45982,0000004_D,00722819_d,0,-1,2017-04-24 20:11:54.191,4
45977,0000004_D,00709419_d,1,-1,2017-04-27 11:59:33.953,4
45983,0000004_D,00424371_d,0,-1,2017-04-27 12:06:58.237,4
45980,0000004_D,00439308_d,1,-1,2017-04-30 03:54:12.752,4
45978,0000004_D,00382341_d,1,-1,2017-04-30 03:56:39.571,4


In [90]:
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.015076,NaN
pd,0.015076,1.000000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,-0.131537,NaN
pd,-0.131537,1.000000,NaN
cv,NaN,NaN,NaN


### Cluster7

In [112]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(5, user_df)
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.481091,NaN
pd,0.481091,1.000000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,-0.128667,NaN
pd,-0.128667,1.000000,NaN
cv,NaN,NaN,NaN


### Cluster5

In [113]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(8, user_df)
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.636416,NaN
pd,0.636416,1.000000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,0.073197,NaN
pd,0.073197,1.000000,NaN
cv,NaN,NaN,NaN


### Cluster6
cartが少(2,192)・pdが多(1530,1144)・cvもあったりなかったり(0,17) /2人

In [125]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(6, user_df)
user_ids

['0016920_D', '0102238_D']

In [127]:
action_df[action_df["user_id"]=="0016920_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
2826,0016920_D,00636526_d,0,-1,2017-04-01 08:14:01.030,6
2827,0016920_D,00366789_d,0,-1,2017-04-01 08:14:55.711,6
1878,0016920_D,00761856_d,0,-1,2017-04-01 08:49:31.996,6
1879,0016920_D,00379512_d,1,-1,2017-04-01 10:08:41.451,6
2825,0016920_D,00298488_d,0,-1,2017-04-01 10:08:42.357,6
1822,0016920_D,00761856_d,1,-1,2017-04-01 11:14:13.935,6
2589,0016920_D,00059147_d,0,-1,2017-04-01 13:12:28.389,6
1880,0016920_D,00626415_d,1,-1,2017-04-01 13:44:02.841,6
1890,0016920_D,00285748_d,1,-1,2017-04-02 03:09:22.839,6
1896,0016920_D,00491613_d,1,-1,2017-04-02 03:09:43.585,6


### Cluster7
cart(mean:23)・pd(mean:13)・cvを全員持つ(mean:1.5)

In [97]:
describe_cluster(7,user_df)

cluster_7


,count,mean,std,min,25%,50%,75%,max
ca,358.0,26.139665,67.134486,0.0,5.0,12.0,22.75,968.0
pd,358.0,13.952514,12.694668,0.0,7.0,10.0,17.00,107.0
cv,358.0,1.522346,0.858833,1.0,1.0,1.0,2.00,5.0


cvもちょくちょくある

In [94]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(7, user_df)
user_ids[0:5]

['0000410_D', '0001211_D', '0001457_D', '0001789_D', '0002804_D']

In [102]:
action_df[action_df["user_id"]=="0001457_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
4696,0001457_D,00176092_d,1,-1,2017-04-01 08:01:21.531,7
4693,0001457_D,00509211_d,0,-1,2017-04-01 14:13:55.849,7
4697,0001457_D,00493161_d,1,-1,2017-04-01 14:23:53.354,7
4698,0001457_D,00427752_d,1,-1,2017-04-01 14:46:23.492,7
4718,0001457_D,00785736_d,0,-1,2017-04-01 15:00:59.999,7
4725,0001457_D,00032217_d,0,-1,2017-04-01 15:04:23.135,7
4723,0001457_D,00087289_d,0,-1,2017-04-01 15:04:51.157,7
4724,0001457_D,00103099_d,0,-1,2017-04-01 15:07:40.106,7
4699,0001457_D,00249301_d,1,-1,2017-04-01 15:12:18.653,7
4701,0001457_D,00166322_d,1,-1,2017-04-01 15:15:08.481,7


In [103]:
cluster_df[cluster_df["user_id"]=="0001457_D"].sort_values("time_stamp")

,user_id,product_id,event_type,ad,time_stamp,cluster
4696,0001457_D,00176092_d,1,-1,2017-04-01 08:01:21.531,7
4693,0001457_D,00509211_d,0,-1,2017-04-01 14:13:55.849,7
4697,0001457_D,00493161_d,1,-1,2017-04-01 14:23:53.354,7
4698,0001457_D,00427752_d,1,-1,2017-04-01 14:46:23.492,7
4718,0001457_D,00785736_d,0,-1,2017-04-01 15:00:59.999,7
4725,0001457_D,00032217_d,0,-1,2017-04-01 15:04:23.135,7
4723,0001457_D,00087289_d,0,-1,2017-04-01 15:04:51.157,7
4724,0001457_D,00103099_d,0,-1,2017-04-01 15:07:40.106,7
4699,0001457_D,00249301_d,1,-1,2017-04-01 15:12:18.653,7
4726,0001457_D,00032217_d,0,-1,2017-04-01 15:14:13.707,7


### Cluster3,10
3
cart(mean:132)・pd(mean:77)・cvは0(一人1)・閲覧を全員50以上する /407

10
cart(mean:50)・pd(mean:33)・cvは0・閲覧を全員20以上する /4431

In [116]:
describe_cluster(3, user_df)

cluster_3


,count,mean,std,min,25%,50%,75%,max
ca,407.0,132.950860,146.277917,0.0,33.5,86.0,188.0,1362.0
pd,407.0,77.248157,30.675159,51.0,60.5,66.0,81.0,300.0
cv,407.0,0.002457,0.049568,0.0,0.0,0.0,0.0,1.0


In [117]:
describe_cluster(10, user_df)

cluster_10


,count,mean,std,min,25%,50%,75%,max
ca,4431.0,50.783119,54.278266,0.0,14.0,36.0,68.0,383.0
pd,4431.0,33.225683,7.239101,21.0,28.0,31.0,37.0,56.0
cv,4431.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [120]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(3, user_df)
user_ids[0:5]

['0000098_D', '0000389_D', '0000536_D', '0000756_D', '0001844_D']

In [122]:
cluster_df[cluster_df["user_id"]=="0000389_D"]

,user_id,product_id,event_type,ad,time_stamp,cluster
67841,0000389_D,00330529_d,0,-1,2017-04-14 07:59:56.503,3
67842,0000389_D,00155966_d,1,-1,2017-04-01 13:06:59.020,3
67843,0000389_D,00788269_d,1,-1,2017-04-02 13:10:15.985,3
67844,0000389_D,00143311_d,1,-1,2017-04-02 13:53:24.635,3
67845,0000389_D,00040867_d,1,-1,2017-04-02 13:42:46.047,3
67846,0000389_D,00616873_d,1,-1,2017-04-02 13:47:59.206,3
67847,0000389_D,00656083_d,1,-1,2017-04-02 15:45:54.224,3
67848,0000389_D,00351008_d,1,-1,2017-04-03 13:30:19.361,3
67849,0000389_D,00087694_d,1,-1,2017-04-04 04:55:12.057,3
67850,0000389_D,00017064_d,1,-1,2017-04-04 08:29:44.911,3


In [115]:
action_df[action_df["user_id"]=="0000098_D"].sort_values(["time_stamp","user_id","product_id"])

,user_id,product_id,event_type,ad,time_stamp,cluster
38901,0000098_D,00135739_d,0,-1,2017-04-01 10:35:04.772,3
38766,0000098_D,00586927_d,0,-1,2017-04-01 10:35:04.772,3
38771,0000098_D,00623191_d,0,-1,2017-04-01 12:47:37.710,3
38792,0000098_D,00307771_d,1,-1,2017-04-02 13:49:32.392,3
38904,0000098_D,00648799_d,0,-1,2017-04-02 14:06:51.908,3
38909,0000098_D,00564453_d,0,-1,2017-04-03 12:16:59.101,3
38907,0000098_D,00247278_d,0,-1,2017-04-03 12:17:28.236,3
38795,0000098_D,00785693_d,1,-1,2017-04-03 12:18:10.856,3
38911,0000098_D,00721921_d,0,-1,2017-04-03 12:20:02.979,3
38794,0000098_D,00029673_d,0,-1,2017-04-03 12:47:03.779,3


In [118]:
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.022126,-0.00242
pd,0.022126,1.000000,0.00052
cv,-0.002420,0.000520,1.00000


user


,ca,pd,cv
ca,1.000000,0.177555,0.048254
pd,0.177555,1.000000,-0.011741
cv,0.048254,-0.011741,1.000000


In [119]:
cluster_df, action_df, user_ids = prepare_cluster_analysis(10, user_df)
count_analysis(cluster_df)

product


,ca,pd,cv
ca,1.000000,0.402851,NaN
pd,0.402851,1.000000,NaN
cv,NaN,NaN,NaN


user


,ca,pd,cv
ca,1.000000,0.090575,NaN
pd,0.090575,1.000000,NaN
cv,NaN,NaN,NaN
